# 第9部　一般化線形モデル

## 3章　一般化線形モデルの評価

### 実装：分析の準備

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
from scipy import stats
# 表示桁数の設定
pd.set_option('display.precision', 3)
np.set_printoptions(precision=3)

# 統計モデルを推定するライブラリ
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
# 表示設定(書籍本文のレイアウトと合わせるためであり、必須ではありません)
np.set_printoptions(linewidth=60)
pd.set_option('display.width', 60)

In [3]:
# データの読み込み
test_result = pd.read_csv('9-2-1-logistic-regression.csv')

# モデル化
mod_glm = smf.glm(formula = 'result ~ hours', 
                  data = test_result, 
                  family=sm.families.Binomial()).fit()

### 実装：ピアソン残差

In [4]:
# ピアソン残差の計算

# 予測された成功確率
pred = mod_glm.predict()
# 応答変数(テストの合否)
y = test_result.result

# ピアソン残差
peason_resid = (y - pred) / np.sqrt(pred * (1 - pred))
peason_resid.head(3)

0   -0.102
1   -0.102
2   -0.102
Name: result, dtype: float64

In [5]:
# ピアソン残差の取り出し
mod_glm.resid_pearson.head(3)

0   -0.102
1   -0.102
2   -0.102
dtype: float64

In [6]:
# ピアソン残差の2乗和
round(np.sum(mod_glm.resid_pearson**2), 3)

84.911

In [7]:
# summary関数でも出力されている
round(mod_glm.pearson_chi2, 3)

84.911

### 実装：deviance残差

#### deviance残差の計算

In [8]:
# deviance残差の計算

# 成功確率の当てはめ値
pred = mod_glm.predict()
# 応答変数(テストの合否)
y = test_result.result

# 合否を完全に予測できたときの対数尤度との差異
resid_tmp = 0 - np.log(stats.binom.pmf(k = y, n = 1, 
                                       p = pred))
# deviance残差
deviance_resid = np.sqrt(
    2 * resid_tmp) * np.sign(y - pred)
# 結果の確認
deviance_resid.head(3)

0   -0.144
1   -0.144
2   -0.144
Name: result, dtype: float64

In [9]:
mod_glm.resid_deviance.head(3)

0   -0.144
1   -0.144
2   -0.144
dtype: float64

#### devianceの計算

In [10]:
# deviance
deviance = np.sum(mod_glm.resid_deviance ** 2)
round(deviance, 3)

68.028

#### 最大化対数尤度からdevianceを計算

In [11]:
# 最大化対数尤度の計算
loglik = sum(np.log(stats.binom.pmf(k=y, n=1, p=pred)))
round(loglik, 3)

-34.014

In [12]:
# 最大化対数尤度の取得
round(mod_glm.llf, 3)

-34.014

In [13]:
# 最大化対数尤度からdevianceを計算
round(2 * (0 - mod_glm.llf), 3)

68.028

In [14]:
# devianceの取得
round(mod_glm.deviance, 3)

68.028